In [1]:
from sleepmate.agent import *
from sleepmate.user import *
from sleepmate.diary import *

In [3]:
!rm -f /Users/cck197/Downloads/virtual_greg/memory.pkl

# Database

In [2]:
from mongoengine import connect, Document, StringField

In [3]:
# Setup and Connection
connect(db="sleepmate", host="localhost", port=27017)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

In [4]:
db_user = User(name="Chris", email="chris@nourishbalancethrive.com")

In [5]:
db_user.save()

<User: User object>

In [4]:
# Querying the User Document
db_user = DBUser.objects(email="chris@nourishbalancethrive.com").first()

In [5]:
db_user.to_json()

'{"_id": {"$oid": "6526d86e6015be5b3434ac13"}, "name": "Chris", "email": "chris@nourishbalancethrive.com"}'

# Meet and Greet

In [3]:
memory = X.load_memory()

load_memory filename='/Users/cck197/Downloads/virtual_greg/memory.pkl'


In [9]:
memory.save_context({"input": f"my name is {user.name}"}, {"output": f"hi {user.name}"})

In [5]:
memory.save_context({"input": f"my email is {user.email}"}, {"output": f"got it"})

In [37]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='my name is Chris'),
  AIMessage(content='hi Chris'),
  HumanMessage(content='hey'),
  AIMessage(content='Hi Chris! How can I assist you today?'),
  HumanMessage(content='Tell me a joke.'),
  AIMessage(content='\n{"setup": "Why did the chicken cross the road?", "punchline": "To get to the other side!"}'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='\n{"setup": "What\'s my name?", "punchline": ""}'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='\n{"setup": "What\'s my name?", "punchline": ""}'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='\n{"name": "Your Name"}'),
  HumanMessage(content="what's the human's name?"),
  AIMessage(content='\n{"name": "John"}'),
  HumanMessage(content="what's the human's name?"),
  AIMessage(content='\n\n{"name": "Chris"}'),
  HumanMessage(content='my email is chris@nourishbalancethrive.com'),
  AIMessage(content='got it')]}

In [59]:
x = X(goals["diary_probe"], memory=memory);

get_agent len(tools)=10


Hello, Chris! How are you feeling today?

# Parsing

In [5]:
from typing import List

from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator

In [6]:
model_name = "text-davinci-003"
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

In [7]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=joke_query)

output = model(_input.to_string())

parser.parse(output)

Joke(setup='Why did the chicken cross the road?', punchline='To get to the other side!')

In [18]:
_input.to_string()

'Answer the user query.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}\n```\nTell me a joke.\n'

# Using

In [38]:
# Define your desired data structure.
class UserParser(BaseModel):
    name: str = Field(description="name")
    email: str = Field(description="email address")

In [39]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=UserParser)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\nPrevious conversation:{chat_history}",
    input_variables=["query", "chat_history"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [40]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

In [41]:
chain = LLMChain(
    llm=model,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [55]:
user_query = "what are the human's details"

In [56]:
output = chain({"query": user_query}); output



> Entering new LLMChain chain...
Prompt after formatting:
Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "name", "type": "string"}, "email": {"title": "Email", "description": "email address", "type": "string"}}, "required": ["name", "email"]}
```
what are the human's details
Previous conversation:[HumanMessage(content='my name is Chris'), AIMessage(content='hi Chris'), HumanMessage(content='hey'), AIMessage(content='Hi Chris! How can I assist you today?'), HumanMessage(content='Tell me a joke.'), AIMessage(co

{'query': "what are the human's details",
 'chat_history': [HumanMessage(content='my name is Chris'),
  AIMessage(content='hi Chris'),
  HumanMessage(content='hey'),
  AIMessage(content='Hi Chris! How can I assist you today?'),
  HumanMessage(content='Tell me a joke.'),
  AIMessage(content='\n{"setup": "Why did the chicken cross the road?", "punchline": "To get to the other side!"}'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='\n{"setup": "What\'s my name?", "punchline": ""}'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='\n{"setup": "What\'s my name?", "punchline": ""}'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='\n{"name": "Your Name"}'),
  HumanMessage(content="what's the human's name?"),
  AIMessage(content='\n{"name": "John"}'),
  HumanMessage(content="what's the human's name?"),
  AIMessage(content='\n\n{"name": "Chris"}'),
  HumanMessage(content='my email is chris@nourishbalancethrive.com'),
  AIMessage(content='got i

In [57]:
parser.parse(output["text"])

UserParser(name='Chris', email='chris@nourishbalancethrive.com')

In [46]:
get_parsed_output(user_query, memory, UserParser)



> Entering new LLMChain chain...
Prompt after formatting:
Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "name", "type": "string"}, "email": {"title": "Email", "description": "email address", "type": "string"}}, "required": ["name", "email"]}
```
what are the human's name and email?
Previous conversation:[HumanMessage(content='my name is Chris'), AIMessage(content='hi Chris'), HumanMessage(content='hey'), AIMessage(content='Hi Chris! How can I assist you today?'), HumanMessage(content='Tell me a joke.'), AIMe

UserParser(name='Chris', email='chris@nourishbalancethrive.com')

In [47]:
up = _

In [49]:
up.dict()

{'name': 'Chris', 'email': 'chris@nourishbalancethrive.com'}

In [50]:
def pydantic_to_mongoengine(pydantic_model):
    fields = {}
    for name, field in pydantic_model.__fields__.items():
        # This is a basic example and only handles str type; you would expand this for other types.
        if field.outer_type_ == str:
            fields[name] = StringField(required=field.required)
        # Add other field type conversions as needed
    return type(pydantic_model.__name__, (Document,), fields)


In [52]:
User = pydantic_to_mongoengine(UserParser)

In [54]:
User.__dict__

mappingproxy({'name': <mongoengine.fields.StringField at 0x146da2190>,
              'email': <mongoengine.fields.StringField at 0x1477e3650>,
              '_meta': {'abstract': False,
               'max_documents': None,
               'max_size': None,
               'ordering': [],
               'indexes': [],
               'id_field': 'id',
               'index_background': False,
               'index_opts': None,
               'delete_rules': None,
               'allow_inheritance': None,
               'collection': 'user_parser',
               'index_specs': []},
              '_is_base_cls': False,
              '_is_document': True,
              '_cached_reference_fields': [],
              '_fields': {'name': <mongoengine.fields.StringField at 0x146da2190>,
               'email': <mongoengine.fields.StringField at 0x1477e3650>,
               'id': <mongoengine.base.fields.ObjectIdField at 0x147674a10>},
              '_db_field_map': {'name': 'name', 'email': 'ema

# New

In [2]:
memory = X.load_memory()

load_memory filename='/Users/cck197/Downloads/virtual_greg/memory.pkl'


In [3]:
user = get_user(memory); user

User(name='Chris', email='chris@nourishbalancethrive.com')

In [6]:
%pdb

Automatic pdb calling has been turned ON


In [3]:
entry = get_sleep_diary_entry(memory); entry

SleepDiaryEntry(quality='very good', in_bed=datetime.datetime(2023, 10, 11, 21, 15), tried_to_fall_asleep='21:15', time_to_fall_asleep='5 mins', times_woke_up='once', time_awake='5 mins', final_wake_up='6:30', out_of_bed='got up right away', medications='melatonin', notes='slept in the van and it was raining')

In [5]:
db_entry = DBSleepDiaryEntry(**{"user": db_user, **entry.dict()})

In [6]:
db_entry.to_json()

'{"user": {"$oid": "6526d86e6015be5b3434ac13"}, "quality": "very good", "in_bed": {"$date": 1697058900000}, "tried_to_fall_asleep": "21:15", "time_to_fall_asleep": "5 mins", "times_woke_up": "once", "time_awake": "5 mins", "final_wake_up": "6:30", "out_of_bed": "got up right away", "medications": "melatonin", "notes": "slept in the van and it was raining"}'

In [8]:
db_entry.save()

<SleepDiaryEntry: SleepDiaryEntry object>

In [12]:
d = db_entry.to_mongo().to_dict(); d

{'_id': ObjectId('6526f58b148ed69172f6a51a'),
 'user': ObjectId('6526d86e6015be5b3434ac13'),
 'quality': 'very good',
 'in_bed': datetime.datetime(2023, 10, 11, 21, 15),
 'tried_to_fall_asleep': '21:15',
 'time_to_fall_asleep': '5 mins',
 'times_woke_up': 'once',
 'time_awake': '5 mins',
 'final_wake_up': '6:30',
 'out_of_bed': 'got up right away',
 'medications': 'melatonin',
 'notes': 'slept in the van and it was raining'}

In [13]:
for k in ("_id", "user"):
    d.pop(k, None)

In [15]:
import json

In [37]:
json_output = json.dumps(d, default=lambda o: o.isoformat() if isinstance(o, datetime) else o); json_output

'{"quality": "very good", "in_bed": "2023-10-11T21:15:00", "tried_to_fall_asleep": "21:15", "time_to_fall_asleep": "5 mins", "times_woke_up": "once", "time_awake": "5 mins", "final_wake_up": "6:30", "out_of_bed": "got up right away", "medications": "melatonin", "notes": "slept in the van and it was raining"}'

In [9]:
json_output = '{"quality": "very good", "in_bed": "2023-10-11T21:15:00", "tried_to_fall_asleep": "21:15", "time_to_fall_asleep": "5 mins", "times_woke_up": "once", "time_awake": "5 mins", "final_wake_up": "6:30", "out_of_bed": "got up right away", "medications": "melatonin", "notes": "slept in the van and it was raining"}'

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate

In [6]:
goals["diary_entry_retrieval"]

'\n        Your goal is to summarise the JSON object to display a sleep diary entry\n        for a given date. Convert ISO 8601 strings to a human readable format.\n        \nInclude sleep efficiency as a percentage. Sleep efficiency is the percentage of\ntime spent asleep while in bed.\n\nTo give the human a rough sense of how their sleep efficiency compares, tell\nthem anything over 85%% is considered normal.\n\n        '

In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=goals["diary_entry_retrieval"]
        ),
        HumanMessagePromptTemplate.from_template("{json}"),
    ]
)

In [10]:
llm = ChatOpenAI(model_name=model_name)
chain = LLMChain(llm=llm, prompt=prompt)
#output = chain({"query": query})
output = llm(prompt.format_messages(json=json_output))

In [11]:
display_markdown(output.content)

Sleep Diary Entry for 11th October 2023:

Quality of Sleep: Very Good

You were in bed by 21:15 and tried to fall asleep immediately. It took you 5 minutes to fall asleep. You woke up once during the night and were awake for 5 minutes. You woke up for the final time at 6:30 and got up right away.

You took melatonin as your medication for the night. 

Notes: You slept in the van and it was raining.

Your sleep efficiency for the night was not provided, but remember, anything over 85% is considered normal.

In [7]:
DBSleepDiaryEntry.objects.delete()

0

In [18]:
DBUser.objects.delete()

1